Annotation으로부터 라벨 값을 전처리하는 코드는 이미 준비됨   
문제는 병렬처리   
파이썬 병렬처러를 위한 학습 진행중   

In [2]:
TRAINVAL_TXT = "dataset/VOCdevkit/VOC2012/ImageSets/Main/trainval.txt"

def get_filenames(txt_path):
    with open(txt_path, "r") as file:
        return [line.replace("\n", "") for line in file.readlines()]


filenames = get_filenames(TRAINVAL_TXT)
print(len(filenames))
filenames[:5]


11540


['2008_000002', '2008_000003', '2008_000007', '2008_000008', '2008_000009']

In [3]:
ANNOTATION_DIR = "dataset/VOCdevkit/VOC2012/Annotations"
import os

label_paths = [
    os.path.join(ANNOTATION_DIR, name + ".xml") for name in filenames
]
label_paths[:5]


['dataset/VOCdevkit/VOC2012/Annotations/2008_000002.xml',
 'dataset/VOCdevkit/VOC2012/Annotations/2008_000003.xml',
 'dataset/VOCdevkit/VOC2012/Annotations/2008_000007.xml',
 'dataset/VOCdevkit/VOC2012/Annotations/2008_000008.xml',
 'dataset/VOCdevkit/VOC2012/Annotations/2008_000009.xml']

In [ ]:
from PIL import Image
import numpy as np
from parse_annotation import get_label

IMAGE_SIZE = (512, 512)
TARGET_DIR = os.path.join(os.curdir, "dataset", "VOCdevkit", "VOC2012", "targets")
import asyncio
from tqdm.asyncio import tqdm

async def save_target(filename, label_path):
    np.save(
        os.path.join(TARGET_DIR, filename + ".npy"),
        get_label(label_path, IMAGE_SIZE),
    )


def init_target():
    tasks = {
        asyncio.create_task(save_target(filename, label_path))
        for filename, label_path in zip(filenames, label_paths)
    }
    pbar = tqdm.tqdm(total=len(tasks))
    for task in asyncio.as_completed(tasks):
        value = task
        pbar.set_description(value)
        pbar.update()


asyncio.run(init_target())


In [5]:
len(os.listdir(TARGET_DIR))

997